**We fetch the data**

We select one city to start with

In [514]:
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
data = pd.read_csv('listings_stockholm.csv.gz', nrows=13815, compression='gzip')

data = data[['accommodates','bedrooms', 'beds', 'review_scores_rating','review_scores_location','review_scores_value','reviews_per_month','property_type','room_type','price']]
# data = data[['accommodates','bedrooms','price']]
data = data.dropna()
data.shape


(2978, 10)

**We define a function to change string features to an integer**

In [515]:
def string_features_to_num(array):
    dictOfWords = { i : np.unique(array)[i] for i in range(0, len(np.unique(array)) ) }
    dictOfWords = {v: k for k, v in dictOfWords.items()}
    
    values = np.zeros(len(array))
    for i in range(len(array)):
        values[i] = dictOfWords.get(array[i])
        
    return values, dictOfWords

In [516]:
def selectPropertyType(type, X, y):
    xlist = []
    ylist = []
    for i in range(X.shape[0]):
        if(X[i,-2] == type):
            xlist.append(X[i,:])
            ylist.append(y[i])
    return np.array(xlist), np.array(ylist)

**Data split into features and target**

We also change the target from a string to a float. 

In [517]:
import re

datanp = np.array(data)
X = datanp[:,:-1]
y = datanp[:,-1]

for i in range(len(y)):
    y[i] = float(re.sub(",", "", (y[i][1:])))

X[:, -2], propertytype_dict = string_features_to_num(X[:, -2])
X[:, -1], roomtype_dict = string_features_to_num(X[:, -1])

print(propertytype_dict)

X, y = selectPropertyType(propertytype_dict.get('Entire rental unit'), X, y)

X.shape

{'Boat': 0, 'Entire bungalow': 1, 'Entire cabin': 2, 'Entire condo': 3, 'Entire cottage': 4, 'Entire guest suite': 5, 'Entire guesthouse': 6, 'Entire home': 7, 'Entire in-law': 8, 'Entire loft': 9, 'Entire place': 10, 'Entire rental unit': 11, 'Entire serviced apartment': 12, 'Entire townhouse': 13, 'Entire vacation home': 14, 'Entire villa': 15, 'Private room': 16, 'Private room in bed and breakfast': 17, 'Private room in boat': 18, 'Private room in casa particular': 19, 'Private room in condo': 20, 'Private room in guest suite': 21, 'Private room in guesthouse': 22, 'Private room in home': 23, 'Private room in hostel': 24, 'Private room in houseboat': 25, 'Private room in loft': 26, 'Private room in rental unit': 27, 'Private room in serviced apartment': 28, 'Private room in townhouse': 29, 'Private room in vacation home': 30, 'Private room in villa': 31, 'Room in aparthotel': 32, 'Room in bed and breakfast': 33, 'Room in boutique hotel': 34, 'Room in hostel': 35, 'Room in hotel': 36

(1257, 9)

**Data split into train/val/test**

In [518]:
X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.25, random_state=42)

**Models imported**

In [519]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR

linear_model = LinearRegression()
svr_model = SVR(C=60, epsilon=0.95)

**Models fit and scored**

We fit the models on the training set and test with the linear regression. 

In [520]:
linear_model.fit(X_train, y_train)
linear_train_score = linear_model.score(X_train, y_train)
linear_val_score = linear_model.score(X_val,y_val)
linear_test_score = linear_model.score(X_test,y_test)




print(linear_train_score, linear_test_score, linear_val_score)

0.22664130585871223 0.3447304867719526 0.1762739709421579


In [521]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_val = scaler.transform(X_val)



In [522]:
from sklearn.svm import NuSVR

svr_model = NuSVR(C=600, nu=0.3)
svr_model.fit(X_train, y_train)
svr_test_score = svr_model.score(X_test, y_test)
print(svr_test_score)

0.39450347059020363


In [523]:
from sklearn.ensemble import RandomForestRegressor

regr = RandomForestRegressor(max_depth=6, random_state=0, n_estimators=100)
regr.fit(X_train, y_train)

print(regr.score(X_test,y_test))


0.15974789256444166


In [524]:
from sklearn.linear_model import Ridge


clf = Ridge(alpha=5)
clf.fit(X_train, y_train)

print(clf.score(X_test,y_test))

0.3463538067194464


In [525]:
# import math
# import pandas as pd
# import tensorflow as tf
# import matplotlib.pyplot as plt
# from tensorflow.keras import Model
# from tensorflow.keras import Sequential
# from tensorflow.keras.optimizers import Adam
# from sklearn.preprocessing import StandardScaler
# from tensorflow.keras.layers import Dense, Dropout
# from tensorflow.keras.losses import MeanSquaredLogarithmicError
# from sklearn.metrics import r2_score

# X_train = pd.DataFrame(X_train)
# X_test = pd.DataFrame(X_test)
# y_train = pd.DataFrame(y_train)
# y_test = pd.DataFrame(y_test)

# def scale_datasets(x_train, x_test):
#     """
#     Standard Scale test and train data
#     Z - Score normalization
#     """
#     standard_scaler = StandardScaler()
#     x_train_scaled = pd.DataFrame(
#         standard_scaler.fit_transform(x_train),
#         columns=x_train.columns
#     )
#     x_test_scaled = pd.DataFrame(
#         standard_scaler.transform(x_test),
#         columns = x_test.columns
#     )
#     return x_train_scaled, x_test_scaled
# x_train_scaled, x_test_scaled = scale_datasets(X_train, X_test)

In [526]:
# hidden_units1 = 160
# hidden_units2 = 480
# hidden_units3 = 256
# learning_rate = 5e-2
# # Creating model using the Sequential in tensorflow
# def build_model_using_sequential():
#   model = Sequential([
#     Dense(hidden_units1, kernel_initializer='normal', activation='relu'),
#     Dropout(0.2),
#     Dense(hidden_units2, kernel_initializer='normal', activation='relu'),
#     Dropout(0.2),
#     Dense(hidden_units3, kernel_initializer='normal', activation='relu'),
#     Dense(1, kernel_initializer='normal', activation='linear')
#   ])
#   return model
# # build the model
# model = build_model_using_sequential()

In [527]:
# # loss function
# msle = MeanSquaredLogarithmicError()
# model.compile(
#     loss=msle, 
#     optimizer=Adam(learning_rate=learning_rate), 
#     metrics=[msle]
# )
# # train the model
# history = model.fit(
#     x_train_scaled.values, 
#     np.asarray(y_train.values).astype('float32'), 
#     epochs=10, 
#     batch_size=10,
#     validation_split=0.2
# )

In [528]:
# def plot_history(history, key):
#   plt.plot(history.history[key])
#   plt.plot(history.history['val_'+key])
#   plt.xlabel("Epochs")
#   plt.ylabel(key)
#   plt.legend([key, 'val_'+key])
#   plt.show()
# # Plot the history
# plot_history(history, 'mean_squared_logarithmic_error')

# X_test['prediction'] = model.predict(x_test_scaled)

# r2_score(y_test, model.predict(x_test_scaled))
